In [53]:
import requests
import hashlib
import time
import pandas as pd
from pandas import json_normalize

secret_token = '696a6f47212d93f88cef71238cfb3d1cd357905b3e34eabd2bbc49124553c129'

# Generate the Unix timestamp
timestamp = int(time.time())

# Construct the query string
query_string = f'time={timestamp}'

# Generate the hash
hash_string = secret_token + str(timestamp)
hash_value = hashlib.sha512(hash_string.encode()).hexdigest()

# Construct the URL with the query string and hash
url = f'https://api.fabdb.net/cards?{query_string}&hash={hash_value}'

parameters = {
    "page": 1,
    "per_page": 100
}

total_pages = 21

# Set the request headers
headers = {
    'Authorization': f'Bearer {secret_token}',
    'Accept': 'application/json'
}


In [54]:
#empty list to hold paginated data
data_records = []

for page in range(1, total_pages + 1):
    parameters['page'] = page
    
    response = requests.get(url, params=parameters)

    # process response
    data = response.json()

    # Extract the relevant data records from the response
    data_records.extend(data['data'])

# Normalize the data before storing it in the dataframe
df = pd.json_normalize(data_records)


In [55]:
df.rename(columns={'stats.resource': 'pitch value'}, inplace=True)

# Map the values in 'pitch value' column to corresponding colors
df['pitch value'] = df['pitch value'].map({1: 'Red', 2: 'Yellow', 3: 'Blue'})

# Rename other columns
df.rename(columns={'stats.life': 'life', 'stats.intellect': 'intellect','stats.attack': 'attack', 'stats.defense': 'defense', 'stats.cost': 'cost'}, inplace=True)
#drop stats column after it's been split
columns_to_drop = ['stats']
df.drop(columns_to_drop, axis=1, inplace=True)


In [56]:
# converting select columns to numeric for statistical analysis
columns_to_convert = ['life', 'attack', 'defense', 'cost', 'intellect']
df[columns_to_convert] = df[columns_to_convert].apply(pd.to_numeric, errors='coerce', downcast='float')
pd.set_option('display.max_rows', None)  # Show all rows
df.head(4)

                                       identifier  \
0                                  eye-of-ophidia   
1                    dash-inventor-extraordinaire   
2                                            dash   
3                             teklo-plasma-pistol   
4                             teklo-foundry-heart   
5                            achilles-accelerator   
6                                     high-octane   
7                                      teklo-core   
8                                maximum-velocity   
9                                 spark-of-genius   
10                              induction-chamber   
11                         pedal-to-the-metal-red   
12                      pedal-to-the-metal-yellow   
13                        pedal-to-the-metal-blue   
14                              pour-the-mold-red   
15                           pour-the-mold-yellow   
16                             pour-the-mold-blue   
17                             aether-sink-yel

In [57]:
#avg numeric stats for all cards
avg_stats = df[['life', 'attack', 'defense','intellect', 'cost']].mean(axis=0,numeric_only=True,skipna=True)
print(avg_stats)

life         22.194805
attack        3.907025
defense       2.524713
intellect     3.983607
cost          1.171671
dtype: float32


In [58]:
# a filtered df showing only Legendary and Mystical Cards with Pitch Value of 3
filtered_df = df[(df['pitch value'] == 'Blue') & ((df['rarity'] == 'M') | (df['rarity'] == 'L'))]

#printing second and third columns and only first 4 rows.
print("Seond and Third Columns/4 rows:")
print(filtered_df.head(4)[['name', 'legality']])


Seond and Third Columns/4 rows:
                    name legality
7             Teklo Core       []
245      Stamp Authority       []
266          Find Center       []
310  Meganetic Shockwave       []


In [59]:
#total nan values for selected columns in blue cards with L or M rarity
nan_counts_blue = filtered_df[['life', 'attack', 'defense', 'cost', 'intellect']].isna().sum()
print(nan_counts)

life         68
attack       55
defense      20
cost          0
intellect    68
dtype: int64


In [60]:
#getting same avg numeric stats for only blue cards that are Legendary or Mystical that pitch for 3
blue_card_stats = filtered_df[['life', 'attack', 'defense','intellect', 'cost']].mean(axis=0,numeric_only=True,skipna=True)
print(blue_card_stats)


life              NaN
attack       2.307692
defense      2.770833
intellect         NaN
cost         1.014706
dtype: float32
